In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, XMLOutputParser
from langchain.output_parsers import YamlOutputParser
from pydantic import BaseModel, Field

In [2]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')

##### Invoking the model for generation:

In [3]:
model = ChatGroq(model="gemma2-9b-it")
print(model.invoke("My name is Namrata.").content)

Hello Namrata, it's nice to meet you!

Is there anything I can help you with today?



##### Creating a chain with prompt and model. Invoking the chain for generation:

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question"),
    ("user","{input}")
])

chain = prompt | model
result = chain.invoke(input={'input':'My name is Namrata.'})
print(result.content)

Hello Namrata! 

It's nice to meet you.  

How can I help you today?  

Since you're asking me as an AI Engineer, are you interested in discussing AI technology, machine learning, or perhaps have a specific project in mind? 

I'm here to assist in any way I can. 😊  




##### Adding different parsers to the chain. Invoking the chain (with parsers) for generation:

StrOutputParser:

In [5]:
str_op_parser = StrOutputParser()
chain = prompt | model | str_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print(result)

LangSmith is an open-source platform developed by AI21 Labs for building and fine-tuning large language models (LLMs).  

Here's a breakdown of what makes LangSmith special:

**Key Features:**

* **User-Friendly Interface:**  LangSmith offers a web-based interface that simplifies the process of working with LLMs, even for users without extensive coding experience.
* **Fine-Tuning Capabilities:**  It allows you to fine-tune pre-trained LLMs on your own datasets, enabling you to customize the model's behavior and performance for specific tasks.
* **Dataset Management:** LangSmith provides tools for managing and annotating your datasets, which is crucial for effective fine-tuning.
* **Experiment Tracking:** It helps you track your fine-tuning experiments, compare different model configurations, and identify the best-performing settings.
* **Community and Collaboration:** As an open-source platform, LangSmith fosters a community of developers and researchers who can share their models, dat

JsonOutputParser:

In [6]:
#OPTION 1: Provide the output format info in the system prompt itself.

prompt_json = ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question. Response should be in JSON object only."),
    ("user","{input}")
])
str_op_parser = StrOutputParser()
chain = prompt_json | model | str_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print(result)

```json
{
  "name": "Langsmith",
  "description": "A platform for building and deploying AI assistants.",
  "features": [
    "Allows users to fine-tune pre-trained language models.",
    "Provides tools for creating conversational interfaces.",
    "Offers a marketplace for sharing and discovering AI assistants.",
    "Supports multiple programming languages and frameworks."
  ],
  "creator": "Cohere"
}
```



In [7]:
#OPTION 2: Using the JsonOutputParser to decide the output format, instead of mentioning it explicitely in the system prompt. Using PromptTemplate

json_op_parser = JsonOutputParser()
output_format = json_op_parser.get_format_instructions()

json_prompt = PromptTemplate(
    template = "You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction} \n {input}",
    input_variables=["input"],
    partial_variables={'format_instruction':output_format}

)
print('Prompt:\n ', json_prompt)

chain = json_prompt | model | json_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print('\n',result)

Prompt:
  input_variables=['input'] input_types={} partial_variables={'format_instruction': 'Return a JSON object.'} template='You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction} \n {input}'

 {'description': 'Langsmith is an open-source framework designed for building and deploying AI-powered applications with large language models (LLMs).', 'key_features': ['Modular and Extensible Design:', 'Supports Multiple LLMs:', 'Streamlined Development Workflow:', 'Built-in Tools for Evaluation and Monitoring:', 'Focus on Responsible AI Practices'], 'benefits': ['Faster Development Cycles:', 'Improved Model Performance:', 'Enhanced Collaboration:', 'Increased Transparency and Accountability'], 'website': 'https://github.com/langsmith-ai/langsmith'}


In [8]:
#OPTION 3: Using the JsonOutputParser to decide the output format, instead of mentioning it explicitely in the system prompt. Using ChatPromptTemplate

json_op_parser = JsonOutputParser()
output_format = json_op_parser.get_format_instructions()

prompt_json = ChatPromptTemplate.from_messages(
    messages=[
            ("system","You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction}"),
            ("user","{input}")
        ]
    )

print('Prompt:\n ', prompt_json)

chain = prompt_json | model | json_op_parser
result = chain.invoke(input={'input':'What is Langsmith?','format_instruction':{output_format}})
print(result)

Prompt:
  input_variables=['format_instruction', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instruction'], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]
{'name': 'Langsmith', 'description': 'Langsmith is an open-source platform for building and deploying AI applications powered by large language models (LLMs).', 'features': ['Modular and composable components for building complex AI workflows', "Support for multiple LLMs, including OpenAI's GPT models and Hugging Face Transformers", 'Tools for data preparation, model training, and evaluation', 'Deployment options for cloud, on-premise, and edge devices', 'Open-source and co

Using Pydantic to standardize the LLM outputs:

In [ ]:
#Using StrOutputParser:

class Product(BaseModel):

    product_name :str = Field(description="name of the product")
    price : int = Field(description="tentative price in USD")

str_parser = StrOutputParser(pydantic_object=Product)

product_prompt = ChatPromptTemplate.from_messages([

    ("system", "You are expert Sales person who has vast knowledge on any product and it's description. You are well versed in product names and their prices."),
    ("user", "{input}")
]

)

chain = product_prompt | model | str_parser
result = chain.invoke(input={'input':'Tell me about the latest iPhone.'})
print('\n',result)


 Let's talk iPhones!  You're probably interested in the **iPhone 15 series**, the latest and greatest from Apple.  They launched in September 2023 and there are four models to choose from, each with unique features:

* **iPhone 15:** The standard model, offering a fantastic balance of performance, features, and price. It boasts a powerful A16 Bionic chip, a vibrant 6.1-inch Super Retina XDR display, and a dual-camera system with impressive low-light capabilities. Starting at **$799**.

* **iPhone 15 Plus:**  For those who want a bigger screen, this model features a spacious 6.7-inch display, the same A16 Bionic chip, and a dual-camera system. It starts at **$899**.

* **iPhone 15 Pro:** This is where things get really exciting. It features a next-generation A17 Pro chip, a stunning 6.1-inch Super Retina XDR display with a ProMotion refresh rate up to 120Hz, and a triple-camera system with a new 48MP main sensor and LiDAR scanner for advanced photography and augmented reality experienc

Using just the pydantic object in the Parser is not enough. We need to provide the formating instructions too in the prompt.

In [ ]:
#Using JsonOutputParser and Prompt Template:

class Product(BaseModel):

    product_name :str = Field(description="name of the product")
    price : int = Field(description="tentative price in USD")

str_parser = JsonOutputParser(pydantic_object=Product)
output_format = str_parser.get_format_instructions()

product_prompt = PromptTemplate(
    template = "You are expert sales person who has vast knowledge on any product and it's price. Provide an answer to the question. \n {format_instruction} \n {input}",
    input_variables=["input"],
    partial_variables={'format_instruction':output_format}

)

chain = product_prompt | model | str_parser
result = chain.invoke(input={'input':'Tell me about the latest iPhone.'})
print('\n',result)


 {'product_name': 'iPhone 15 Pro Max', 'price': 1099}


In [16]:
#Using JsonOutputParser and ChatPrompt Template:

class Product(BaseModel):

    product_name :str = Field(description="name of the product")
    price : int = Field(description="tentative price in USD")

str_parser = JsonOutputParser(pydantic_object=Product)
output_format = str_parser.get_format_instructions()

product_prompt = ChatPromptTemplate.from_messages(
    messages=[
            ("system","You are expert sales person who has vast knowledge on any product and it's price. Provide an answer to the question. \n {format_instruction}"),
            ("user","{input}")
        ]
    )

chain = product_prompt | model | str_parser
result = chain.invoke(input={'input':'Tell me about the latest iPhone.', 'format_instruction':output_format})
print('\n',result)


 {'product_name': 'iPhone 15 Pro Max', 'price': 1099}


Using Pydantic we can thus standardize the output of the LLMs. The output will always come using only the information required according to the pydantic class object passed to the parser.

Using Pydantic model on YamlOutputParser:

In [ ]:
#Using YamlOutputParser and Prompt Template:

class Product(BaseModel):

    product_name :str = Field(description="name of the product")
    price : int = Field(description="tentative price in USD")

str_parser = YamlOutputParser(pydantic_object=Product)
output_format = str_parser.get_format_instructions()

product_prompt = PromptTemplate(
    template = "You are expert sales person who has vast knowledge on any product and it's price. Provide an answer to the question. \n {format_instruction} \n {input}",
    input_variables=["input"],
    partial_variables={'format_instruction':output_format}

)

chain = product_prompt | model | str_parser
result = chain.invoke(input={'input':'Tell me about the latest iPhone.'})
print('\n',result)


 product_name='iPhone 15 Pro Max' price=1099
